In [1]:
import h5py
import pandas as pd
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Resizing, Dropout, BatchNormalization, Activation, Add, GlobalAveragePooling2D, Input, Reshape, Conv2DTranspose, Cropping2D

from keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from tools import define_cnn, extract_features, build_model, predict_with_model, set_seed, build_model_and_predict
import warnings
warnings.filterwarnings('ignore')

c:\Users\jakub\anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
# load clean gdp data, keep only year, region and real_gdp columns
poland = pd.read_csv("data/tabular_data_poland.csv")

# get the data for 2021, 2022 and before 2022
poland_2022 = poland[poland["year"] == 2022]
poland = poland[poland["year"].astype(int) < 2022]
poland_2021 = poland[poland["year"] == 2021]
poland_2021.reset_index(drop=True, inplace=True)
poland_2022.reset_index(drop=True, inplace=True)
poland.reset_index(drop=True, inplace=True)

# Initialise a three dimensional array to store the images with the shape (number of images, height, width, channels)
X = np.zeros((len(poland), 609, 911, 1))
y = np.zeros(len(poland))

# load the images
for i in range(len(poland)):

    # get year, region, and gdp
    year = poland["year"][i]
    region = poland["region"][i]
    gdp_value = poland["real_gdp"][i]

    # load the image
    file_name = f"{year}_{region}_hq.h5"
    file_path = f"data/annual_region_images/{file_name}"
    
    with h5py.File(file_path, 'r') as annual_region:
        allangle_snow_free = annual_region["AllAngle_Composite_Snow_Free"][:]

    # add the values
    y[i] = gdp_value
    X[i, :, :, 0] = allangle_snow_free

# normalise the images and gdp data
maximum_x = X.max()
X = X / maximum_x

maximum_y = y.max()
y = y / maximum_y

# get indices for observations with  year = 2021: this is the test set
test_indices = np.where(poland["year"].astype(int) == 2021)[0]
train_indices = np.where(poland["year"].astype(int) != 2021)[0]

# get the train and test sets
X_train, y_train, X_test, y_test = X[train_indices], y[train_indices], X[test_indices], y[test_indices]

# get the prediction data
X_pred = np.zeros((len(poland_2022),  609, 911, 1))
for i in range(len(poland_2022)):

    year = poland_2022["year"][i]
    region = poland_2022["region"][i]

    file_name = f"{year}_{region}_hq.h5"
    file_path = f"data/annual_region_images/{file_name}"

    with h5py.File(file_path, 'r') as annual_region:
        allangle_snow_free = annual_region["AllAngle_Composite_Snow_Free"][:]
    
    X_pred[i, :, :, 0] = allangle_snow_free

X_pred = X_pred / maximum_x

In [5]:
# define the grid of parameters: network architecture and number of features
param_grid_cnn = {
    'n_features': [4],
    'n_conv': [3],
    'n_dense': [2],
}

# df to store the results
cnn_results = pd.DataFrame(columns = ["n_features", "n_conv", "n_dense", "mae", "mpe", "gdp_change"])

# train first stage, neural network
for n_features in param_grid_cnn['n_features']:
    for n_conv in param_grid_cnn['n_conv']:
        for n_dense in param_grid_cnn['n_dense']:
            set_seed(0)
            model = define_cnn(n_features = n_features, n_conv = n_conv, n_dense = n_dense, input_shape = (609, 911, 1), res_x = 305, res_y = 455)
            model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])
            model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2)

            # extract features
            poland_stage_2, poland_2021, poland_2022 = extract_features(model, poland, poland_2021, poland_2022, X_train, X_pred, X_test, n_features)
            del model

            # train second stage, xgboost
            train_data = poland_stage_2
            test_data = poland_2021
            prediction_data = poland_2022
            pre_war_data = pd.concat([train_data, test_data])
            selected_columns =  ["feature_" + str(i) for i in range(1, n_features+1)]

            # param_grid_xgb = {
            #     'eta': [0.01, 0.1, 0.2, 0.3],
            #     'gamma': [10, 20, 50, 100],
            #     'max_depth': [4, 6, 8, 10],
            #     'min_child_weight': [3, 4, 5, 6],
            #     'random_state': [0] 
            # }

            param_grid_rf = {
                    # 'n_estimators': [50, 100, 200, 300],
                    # 'max_depth': [5, 10, 15, 20],  # Maximum depth of the tree
                    # 'min_samples_split': [2, 4, 6, 8],  # Minimum number of samples required to split an internal node
                    # 'min_samples_leaf': [2, 4, 6, 8],  # Minimum number of samples required to be at a leaf node
                    'random_state': [0],  # Ensures reproducibility
            }

            # mae, mpe, gdp_change, gdp_predictions = build_model_and_predict(pre_war_data, prediction_data, selected_columns, "xgboost", param_grid_xgb, log_transform = False, scale = False, total_metrics = True)
            mae, mpe, gdp_change, gdp_predictions, best_params, metrics = build_model_and_predict(pre_war_data, prediction_data, selected_columns, "random_forest", param_grid_rf, log_transform = False, scale = False, total_metrics = True)

            # add the results to the dataframe
            new_row = pd.DataFrame([{
                "n_features": n_features, 
                "n_conv": n_conv, 
                "n_dense": n_dense, 
                "mae": mae, 
                "mpe": mpe, 
                "gdp_change": gdp_change
            }])
            
            cnn_results = pd.concat([cnn_results, new_row], ignore_index=True)

            print("Results for n_features = ", n_features, "n_conv = ", n_conv, "n_dense = ", n_dense)

Epoch 1/50
2/2 [==============================] - 4s 1s/step - loss: 0.0970 - mae: 0.2350 - val_loss: 0.0334 - val_mae: 0.1405
Epoch 2/50
2/2 [==============================] - 1s 378ms/step - loss: 0.0708 - mae: 0.1872 - val_loss: 0.0220 - val_mae: 0.1170
Epoch 3/50
2/2 [==============================] - 1s 366ms/step - loss: 0.0468 - mae: 0.1549 - val_loss: 0.0197 - val_mae: 0.1268
Epoch 4/50
2/2 [==============================] - 1s 359ms/step - loss: 0.0456 - mae: 0.1542 - val_loss: 0.0177 - val_mae: 0.1122
Epoch 5/50
2/2 [==============================] - 1s 357ms/step - loss: 0.0472 - mae: 0.1427 - val_loss: 0.0150 - val_mae: 0.1091
Epoch 6/50
2/2 [==============================] - 1s 377ms/step - loss: 0.0436 - mae: 0.1444 - val_loss: 0.0232 - val_mae: 0.1377
Epoch 7/50
2/2 [==============================] - 1s 355ms/step - loss: 0.0438 - mae: 0.1588 - val_loss: 0.0142 - val_mae: 0.1114
Epoch 8/50
2/2 [==============================] - 1s 363ms/step - loss: 0.0426 - mae: 0.1415 

In [6]:
metrics

,year,mae,mpe
0,2012,10237.892925,10.481053
1,2013,13321.406610,11.683564
2,2014,6841.567108,7.001205
3,2015,6837.188868,5.874440
4,2016,6737.654020,5.648883
5,2017,6485.316617,4.177963
6,2018,9641.126700,8.088170
7,2019,7434.927743,7.092513
8,2020,12448.534150,7.374180
9,2021,15260.343444,11.800849


In [6]:
poland_2022

,region,year,real_gdp,feature_1,feature_2,feature_3,feature_4
0,Dolnoslaskie,2022,198097.229299,0.329895,0.333257,0.336135,0.322477
1,Kujawsko-Pomorskie,2022,99759.927578,0.168155,0.157030,0.147060,0.164215
2,Lodzkie,2022,135546.540084,0.237761,0.239311,0.236077,0.231868
3,Lubelskie,2022,85552.367159,0.191320,0.184219,0.178757,0.196013
4,Lubuskie,2022,48706.499454,0.082593,0.070241,0.067090,0.079671
5,Malopolskie,2022,183245.329912,0.294531,0.299047,0.298944,0.290297
6,Mazowieckie,2022,543002.219002,0.929369,0.953755,0.964121,0.916522
7,Opolskie,2022,45123.339120,0.089333,0.078340,0.076464,0.093409
8,Podkarpackie,2022,88230.432674,0.144655,0.133580,0.123981,0.144715
9,Podlaskie,2022,50599.773440,0.082383,0.070954,0.066912,0.080228


In [10]:
cnn_results

,n_features,n_conv,n_dense,mae,mpe,gdp_change
0,4,3,2,9524.595818,7.922282,-6.143095
